Multiple Myeloma data challenge
============================
Technical test for Data Scientist (DS) position in the Computational Biology team

Katarzyna Wreczycka 17.10.2022




Clinical context
-------------------
Multiple Myeloma (MM) is a type of bone marrow cancer. Treatment for MM involves
combinations of drugs over multiple cycles. There is huge heterogeneity in treatment
response with some individuals being non-responders and some patients remaining well for
some time before a relapse. A better characterization of patients who relapse early can
influence the treatment options and combinations.

In this test, we propose to develop a model for predicting the risk of dying or relapsing of
newly diagnosed multiple myeloma patients from baseline clinical and expression data.


Data
--------------------------
The data for this test are extracted from an old Synapse Dream Challenge
(https://www.synapse.org/#!Synapse:syn6187098/wiki/401884) .
It consists of clinical data, gene expression data and follow-up for newly diagnosed Multiple
Myeloma patients extracted from the MMRF CoMMpass IA9 study. In the data, newly
diagnosed MM patients are classified as High Risk (HR) when they relapse or die before 18
months.
To access the data, you first need to create an account and download the following files:
- Expression data:
MMRF_CoMMpass_IA9_E74GTF_Salmon_entrezID_TPM_hg19.csv
(https://www.synapse.org/#!Synapse:syn10573789)
[notice that the first column gives Entrez IDs for genes]
- Clinical data and labels:
sc3_Training_ClinAnnotations.csv
(https://www.synapse.org/#!Synapse:syn9926878)
- Explanation of the clinical and label annotations:
Harmonized_Clinical_Dictionary
(https://www.synapse.org/#!Synapse:syn9744732)



Goal
--------------------------------------
The purpose of this technical test is to develop a model for predicting the risk of fast dying or
relapsing of newly diagnosed multiple myeloma patients (using the High Risk label
HR_FLAG).The evaluation will mostly rely on the way you approach the problem: pre-analysis,
preprocessing strategy, choice of modelization and coding skills.
The code should be developed so that the model can be applied to an external validation
dataset. You will send your code (Notebook or script) along with a small report to interpret
the model and put it in MM context (the use of the literature is clearly welcome).
Your model can be developed in Python or R with a small README to explain how to apply
it to external data.

You can use external knowledge/data to develop the model. Please add all the requirements
for libraries that should be installed to make it run.

If not used to survival analysis, the candidate can consider a simplified version in which it
can assumed that no censored patients will be present in the external validation dataset.

(Obvious) suggestion: OS and PFS related variables are also labels and not features:
HR_FLAG is defined as OS or PFS < 18 months (taking into account censoring).

--------------------------
-----------------------------

OS = overall survival

PFS = progression-free survival = The length of time during and after the treatment of a disease, such as cancer, that a patient lives with the disease but it does not get worse. In a clinical trial, measuring the progression-free survival is one way to see how well a new treatment works.

Censoring = a type of missing data problem unique to survival analysis. This happens when you track the sample/subject through the end of the study and the event never occurs. This could also happen due to the sample/subject dropping out of the study for reasons other than death, or some other loss to followup. The sample is censored in that you only know that the individual survived up to the loss to followup, but you don’t know anything about survival after tha


high risk = defined as disease progression or death prior to 18 months from diagnosis


in the publication (https://www.nature.com/articles/s41375-020-0742-z) they say something about 
https://blog.datadive.net/selecting-good-features-part-i-univariate-selection/
"The top-performing model implemented a wisdom of the crowd approach, utilizing clinical features and published myeloma signatures that summarize the expression of gene sets. The second-place “Stanford University Go” (SUGO) model instead included individual genes as features, utilizing a univariate-based feature selection approach to identify genes to include in their model"

(REE-laps) The return of a disease or the signs and symptoms of a disease after a period of improvement


expression data is TPM (transcripts per million).


The International Staging System (ISS) for multiple myeloma defines 3 subgroups with differing overall survival: Stage I- 62 months. Stage II- 44 months. Stage III- 29 months.



Results
-----------------------


In [ ]:
# Load libraries

suppressPackageStartupMessages(library(tidyverse));
suppressPackageStartupMessages(library(data.table));
suppressPackageStartupMessages(library(org.Hs.eg.db));
suppressPackageStartupMessages(library(stats));

# visualization
suppressPackageStartupMessages(library(ggfortify)); 
suppressPackageStartupMessages(library(ggplot2));


suppressPackageStartupMessages(library(glmnet));
suppressPackageStartupMessages(library(survival));
suppressPackageStartupMessages(library(ggplot2));
suppressPackageStartupMessages(library(maxstat))


library(stats)
library(ggfortify); 
library(ggplot2)

In [ ]:
# load data
clinical_data = read.csv("./Harmonized_Clinical_Dictionary.csv", sep=",", header=TRUE,fill = TRUE, stringsAsFactors = FALSE)
expression_data = read.csv("./MMRF_CoMMpass_IA9_E74GTF_Salmon_entrezID_TPM_hg19.csv", sep=",", header=TRUE,stringsAsFactors = FALSE,row.names=1)
annot_data = read.csv("./sc3_Training_ClinAnnotations.csv", sep=",", header=TRUE, stringsAsFactors = FALSE)

In [ ]:
# Convert the row names to entrez ids
#library("AnnotationDbi")
#library("org.Hs.eg.db")
#geneSymbols <- mapIds(org.Hs.eg.db, keys=rownames(expression_data), 
#                      column="SYMBOL", keytype="ENTREZID", 
#                      multiVals="first")
#head(geneSymbols)

In [ ]:

print(clinical_data[which(clinical_data$names=="Study"),]$description)
table(annot_data$Study)


print("PatientType")
table(annot_data$PatientType)


print(clinical_data[which(clinical_data$names=="D_Gender"),]$description)
table(annot_data$D_Gender)

print(clinical_data[which(clinical_data$names=="D_Age"),]$description)
summary(annot_data$D_Age)


print("HR_FLAG")
table(annot_data$HR_FLAG)


print(clinical_data[which(clinical_data$names=="D_ISS"),]$description)
table(annot_data$D_ISS)

print(clinical_data[which(clinical_data$names=="D_OS_FLAG"),]$description)
table(annot_data$D_OS_FLAG)


print(clinical_data[which(clinical_data$names=="D_OS"),]$description)
summary(annot_data$D_OS)

print(clinical_data[which(clinical_data$names=="D_OS_FLAG"),]$description)
table(annot_data$D_OS_FLAG)


print(clinical_data[which(clinical_data$names=="D_PFS"),]$description)
summary(annot_data$D_PFS)


In [ ]:
##############################

print(head(annot_data$D_ISS))
a = as.factor(as.character(annot_data$D_ISS))
levels(a) <- list("I"="1","II"="2","III"="3")
annot_data$D_ISS = a


Data preprocessing
---------------------

In [ ]:
options(repr.plot.width=20, repr.plot.height=10)


# we organized the crowd sourced Multiple Myeloma DREAM Challenge, 
# focusing on predicting high risk, defined as disease progression 
# or death prior to 18 months from diagnosis. 



#----------------- Preprocess the input data

edata = expression_data


# make sure that I all samples that I am going to analyse 
# have also labels
colnames(edata) = gsub("_1_BM","",colnames(edata))
intersect_labels = intersect(colnames(edata),annot_data$Patient)
edata_m = edata[, match(intersect_labels, colnames(edata), nomatch = 0)]
annot_data_m = annot_data[match(intersect_labels, annot_data$Patient, nomatch = 0), ]
#sum(colnames(edata_m)==annot_data_m$Patient) == length(colnames(edata_m))

# TODO: color-code it by HR FLAF, and labels 45 degrees
bp <- boxplot(edata_m[,1:50],outline=FALSE,col="cornflowerblue", xaxt = "n", yaxt = "n")
tick <- seq_along(bp$names)
axis(1, at = tick, labels = FALSE)
text(tick, par("usr")[3] - 0.3, bp$names, srt = 45, xpd = TRUE)

#library(reshape2)
#mt = melt(edata_m[,1:50])
#ggplot(mt, aes(x = variable, y = value)) +
#  geom_boxplot()
#print(head(edata_m[,1:50]))



In [ ]:
#----------------- Filtering data and scaling

# convert a value to a Standard Score ("z-score")

# transform the expression data to Z scores
#edata_sc <- t(scale(t(edata_m)))

# Since taking a log seems to work to tame the extreme values, 
# we do that below and also add 1 pseudo-count to be able to deal with 0 values
gexp=log10(edata_m+1)

# transpose the data set
tgexp <- t(gexp)

library(caret)
# remove near zero variation for the columns at least 85% of the values are the same
# this function creates the filter but doesn't apply it yet
nzv=preProcess(tgexp, method="nzv", uniqueCut = 15)

# apply the filter using "predict" function
# return the filtered dataset and assign it to nzv_tgexp variable
nzv_tgexp=predict(nzv,tgexp)

#anyNA(nzv_tgexp) # check if there are NA values


#gene.quant <- apply(beer.exprs.sample, 1, quantile, probs = 0.75)
#dat.filt <- beer.exprs.sample[gene.quant > log2(100),]
#gene.iqr <- apply(dat.filt, 1, IQR)
#dat.filt <- as.matrix(dat.filt[gene.iqr > 0.5,])
#dat.filt <- t(dat.filt)
#dat.filt <- data.frame(dat.filt)



In [ ]:
# check if there are NA values in the expression matrix
if(anyNA(nzv_tgexp)) print("There are NAs in the expression matrix")

# show 
nzv_gexp = t(nzv_tgexp)
boxplot(nzv_gexp[,1:50],outline=FALSE,col="cornflowerblue")



Clustering
---------------------





In [ ]:
# Let’s select the top 1000 most variable genes among the samples.
options(repr.plot.width=20, repr.plot.height=10)


library(pheatmap)

#compute the variance of each gene across samples
V <- apply(nzv_gexp, 1, var)
#sort the results by variance in decreasing order 
#and select the top 100 genes 
selectedGenes <- names(V[order(V, decreasing = T)][1:1000])


#Now we can quickly produce a heatmap where samples and genes are clustered.
#library(pheatmap)
#pheatmap(nzv_gexp[selectedGenes,], scale = 'row', show_rownames = FALSE)


annot_sel_col <- c("D_Age", 
                   "D_Gender", 
                   "D_OS",
                   "D_PFS", 
                   "D_ISS", 
                   "HR_FLAG"
                  )
colData <- annot_data_m[which(colnames(annot_data_m) %in% annot_sel_col)]
rownames(colData) = colnames(nzv_gexp)

#print(head(nzv_gexp[selectedGenes,]))
#print(head(colData))

library(dichromat)
colramp = colorRampPalette(c(3,"white",2))(9)

pheatmap(nzv_gexp[selectedGenes,], 
         scale = 'row', 
         show_rownames = FALSE, 
         show_colnames = FALSE, 
         annotation_col = colData,
         col=colramp,
         main="All patients")


# Plot newly classified patients that relapsed or died before 18 months (including censored)
pheatmap(nzv_gexp[selectedGenes,which(colData$D_PFS<547.501)], 
         scale = 'row', 
         show_rownames = FALSE, 
         show_colnames = FALSE, 
         annotation_col = colData[which(colData$D_PFS<547.501),],
         col=colramp,
         main="High-risk patients")

pheatmap(nzv_gexp[selectedGenes,which(colData$D_PFS>=547.501)], 
         scale = 'row', 
         show_rownames = FALSE, 
         show_colnames = FALSE, 
         annotation_col = colData[which(colData$D_PFS>=547.501),],
         col=colramp,
        main="Low-risk patients")


# https://rviews.rstudio.com/2017/09/25/survival-analysis-with-r/
# https://www.biostars.org/p/344233/
# https://cran.r-project.org/web/packages/glmnet/vignettes/Coxnet.pdf
# https://compgenomr.github.io/book/gene-expression-analysis-using-high-throughput-sequencing-technologies.html
# https://medium.com/towards-data-science/five-tips-on-survival-analysis-for-a-data-scientist-ba9fd97cbb2d
# https://www.randomforestsrc.org/articles/getstarted.html
# https://bioconnector.github.io/workshops/r-survival.html#tcga
# https://rviews.rstudio.com/2017/09/25/survival-analysis-with-r/
# https://www.biostars.org/p/344233/

In [ ]:
options(repr.plot.width=7, repr.plot.height=7)


# PCA
# Let’s make a PCA plot to see the clustering of replicates as a scatter plot in two dimensions 
#transpose the matrix 
#M <- t(nzv_gexp[selectedGenes,])
#M <- t(nzv_gexp)

#nzv_gexp_centered2 = t(nzv_gexp) - colMeans(nzv_gexp)

# transform the counts to log2 scale 
#M <- log2(M + 1)
#compute PCA 
pcaResults <- prcomp(t(nzv_gexp), center = TRUE)

#plot PCA results making use of ggplot2's autoplot function
#ggfortify is needed to let ggplot2 know about PCA data structure.

annot_data_m$D_ISS = as.factor(annot_data_m$D_ISS)

#autoplot(pcaResults)

autoplot(pcaResults, data = annot_data_m, colour = 'D_Gender')
autoplot(pcaResults, data = annot_data_m, colour = 'D_ISS')
autoplot(pcaResults, data = annot_data_m, colour = 'D_Age')
autoplot(pcaResults, data = annot_data_m, colour = 'HR_FLAG')



In [ ]:
# Modify annotation table
y = annot_data_m
y$time = y$D_PFS
y$status = y$D_PFS_FLAG
y$high.risk <- unlist(apply(y[, c("status", "time")], 1,
                                function(row) ifelse((row[1] == 1) && (row[2] < 18*(365.25/12)), "high", "low")))


In [ ]:
#print(pcaResults$)
#dim(pcaResults$x)
#print(dim(annot_data_m))


Correct gene expression data using the ComBat algorithm
------------------------------------------------

Looking at the results of the hierarchical custering and PCA above, I'll try to remove batch effects and other unwanted variation.



In [ ]:
#edata = expression_data[rowMeans(expression_data) > 100,]
# to be ona scale thats easier to work with
#edata = log2(edata + 1)
# if we dont center the data, if we dont remove rowmeans of the center, the column means of data
# then the first singual value will always be the mean level
# since that will always explain the most variation in genomics experiment
# we want to see variation between sample or between genes
#edata_centered = edata - rowMeans(edata) 
#svd1 = svd(edata_centered)
#names(svd1)

In [ ]:
# try KNN, Support Vector Machines and Ensemble Methods like Random Forests or Gradient Boosting

Baseline models: age, ISS, and age + ISS
---------------------------------------------------------

Baseline models were Cox proportional hazards models where PFS was the response variable and age, ISS, or both age and ISS were explanatory variables. Continuous risk predictions scores of each sample were thresholded to create high risk classifications. The threshold of a given model was calculated by generating sample level  predictions in the training data. The threshold was defined as the cutpoint that maximized the logrank statistic comparing above and below the threshold to their true high-risk status. Continuous predictions for validation data were computed by applying the above trained Cox proportional hazards model to them. These values were dichotomized into high and low risk according to whether the prediction was above or below, respectively, the threshold.

The baseline predictors were Cox proportional hazard models where progression free survival was modeled on age or ISS or age and ISS together. As comparators these models represent a reasonable lower bound on expected performance of high-risk classifiers. 



In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

# Here is a survival analysis based on the one of the top models, 
# I am not sure though that it works well

coxph.model = function(y, clinical.covariates){

    surv.formula <- as.formula(paste("Surv(time, status)", "~",
                                     paste(clinical.covariates, collapse = " + "), sep = " "))
    cph.fit <- coxph(surv.formula, data = y)
    
    pred <- unname(predict(cph.fit, newdata = y, type = "risk"))
    y$pred <- pred

    ## Calculate a threshold for the Cox proportional hazards model                                                                                                                     
    mt.formula <- as.formula(paste("Surv(time, status)", "~", "pred", sep = " "))
    mt <- maxstat.test(mt.formula, data=data.frame(y), smethod="LogRank", pmethod="none")
    print(mt$estimate)
    
    return(
        list(y = y, 
             fitted.model = cph.fit, 
             threshold = as.numeric(mt$estimate))
        )
    
}


############## age
                            
res = coxph.model(y, "D_Age")
threshold <- res$threshold
y$pred = res$y$pred
y$inferred.high.risk <- y$pred > threshold

                            
fisher.test(y$high.risk, y$inferred.high.risk)
g <- ggplot(data = y, aes(x = high.risk, y = pred))
g <- g + geom_boxplot()  +   geom_jitter()  + ggtitle("Age") + ylab("Prediction") + xlab("Defined risk")             
g     
    
                            
cox_fit <- survfit(res$fitted.model)
#plot(cox_fit, main = "cph model", xlab="Days")
autoplot(cox_fit) + ggtitle("Cox proportional hazard model: Age") + xlab("Days") + ylab("Survival")
                            
                            

In [ ]:
############## ISS
                            
res = coxph.model(y, "D_ISS")
threshold <- res$threshold
y$pred = res$y$pred
y$inferred.high.risk <- y$pred > threshold

                            
fisher.test(y$high.risk, y$inferred.high.risk)
g <- ggplot(data = y, aes(x = high.risk, y = pred))
g <- g + geom_boxplot()  +   geom_jitter()  + ggtitle("ISS") + ylab("Prediction") + xlab("Defined risk")             
g    

cox_fit <- survfit(res$fitted.model)
#plot(cox_fit, main = "cph model", xlab="Days")
autoplot(cox_fit) + ggtitle("Cox proportional hazard model: ISS") + xlab("Days") + ylab("Survival")
     
  

In [ ]:
options(repr.plot.width=8, repr.plot.height=8)

############## age + ISS  
                            
                            
res = coxph.model(y, "D_Age + D_ISS")
threshold <- res$threshold
y$pred = res$y$pred
y$inferred.high.risk <- y$pred > threshold

                            
fisher.test(y$high.risk, y$inferred.high.risk)
g <- ggplot(data = y, aes(x = high.risk, y = pred))
g <- g + geom_boxplot()  +   geom_jitter()  + ggtitle("D_Age + D_ISS") + ylab("Prediction") + xlab("Defined risk")             
g 


cox_fit <- survfit(res$fitted.model)
#plot(cox_fit, main = "cph model", xlab="Days")
autoplot(cox_fit) + ggtitle("Cox proportional hazard model: Age+ISS") + xlab("Days") + ylab("Survival")
     

In [ ]:
library(survAUC)

head(y)
#res = coxph.model(y, "D_Age + D_ISS")
#threshold <- res$threshold
#y$pred = res$y$pred
#y$inferred.high.risk <- y$pred > threshold

# 5-fold cross-validation repeated 5 times
set.seed(123)
kfold=5
n=nrow(y)
n2sample = floor(n / 5)

indx.train = sample(n, n2sample)
indx.test = c(1:n)[-indx.train]
ytest = y[indx.test,]
ytrain=y[indx.train,]
#length(indx.train)
#length(indx.test)
#dim(ytrain)
#dim(ytest)


intauc_list = c()
for (i in 1:100){
    
   
    clinical.covariates = "D_Age + D_ISS"
    surv.formula <- as.formula(paste("Surv(time, status)", "~",
                                     paste(clinical.covariates, collapse = " + "), sep = " "))
    train.fit <- coxph(surv.formula, data = ytrain, method="breslow",x=TRUE, y=TRUE)
    
    lp <- predict(train.fit)
    lpnew <- predict(train.fit, newdata=ytest)
    Surv.rsp <- survival::Surv(ytrain$time, ytrain$status)
    Surv.rsp.new <- survival::Surv(ytest$time, ytest$status)
    times <- seq(from=365, to=730, by=7) # from 12 months to 2 years every week
    
    AUC_CD <- AUC.cd(Surv.rsp, Surv.rsp.new, lp, lpnew, times)
    
    intauc=IntAUC(AUC_CD$auc, AUC_CD$times, runif(length(times),0,1), 
                  median(times), auc.type="cumulative")
    intauc_list = c(intauc_list, intauc)

}



In [ ]:
print(intauc_list)



In [ ]:
train.fit  <- survival::coxph(survival::Surv(futime, fustat) ~ age,
                              x=TRUE, y=TRUE, method="breslow", data=TR)

lp <- predict(train.fit)
lpnew <- predict(train.fit, newdata=TE)
Surv.rsp <- survival::Surv(TR$futime, TR$fustat)
Surv.rsp.new <- survival::Surv(TE$futime, TE$fustat)
times <- seq(10, 1000, 10)                  
times

AUC_CD <- AUC.cd(Surv.rsp, Surv.rsp.new, lp, lpnew, times)

AUC_CD

runif(length(times),0,1)
IntAUC(AUC_CD$auc, AUC_CD$times, runif(length(times),0,1), 
       median(times), auc.type="cumulative")


In [ ]:
library(survAUC)

data(cancer,package="survival")
TR <- ovarian[1:16,]
TE <- ovarian[17:26,]
train.fit  <- survival::coxph(survival::Surv(futime, fustat) ~ age,
                              x=TRUE, y=TRUE, method="breslow", data=TR)

lp <- predict(train.fit)
lpnew <- predict(train.fit, newdata=TE)
Surv.rsp <- survival::Surv(TR$futime, TR$fustat)
Surv.rsp.new <- survival::Surv(TE$futime, TE$fustat)
times <- seq(10, 1000, 10)                  
times

AUC_CD <- AUC.cd(Surv.rsp, Surv.rsp.new, lp, lpnew, times)

AUC_CD

runif(length(times),0,1)
IntAUC(AUC_CD$auc, AUC_CD$times, runif(length(times),0,1), 
       median(times), auc.type="cumulative")


summary(TE$futime)

# head(ovarian)
#	futime	fustat	age	resid.ds	rx	ecog.ps#
#	<dbl>	<dbl>	<dbl>	<dbl>	<dbl>	<dbl>
#1	59	1	72.3315	2	1	1
#2	115	1	74.4932	2	1	1
#3	156	1	66.4658	2	1	2
#4	421	0	53.3644	2	2	1
#5	431	1	50.3397	2	1	1
#6	448	0	56.4301	1	1	2

#The training dataset was split using repeated k-fold cross-validation 
#(specifically, 5 folds repeated 5 times) and used to calculate the average iAUC 
#over folds and repeats. This is the reference out-of-sample performance of the model.
#Then the observed values of one of the variables were permuted and the predictions
#and average iAUC were recalculated. The difference between the reference performance
#and the new performance after permuting the variable defined the importance of that 
#variable. This procedure is then repeated for all variables in the model.


#myModel = cv.glmnet(data.matrix(modelData), 
#                    modelData$ACTION,
#                    family = "binomial",
#                    type.measure = "auc",
#                    nfolds = 5,
#                    alpha = 1)
# https://stats.stackexchange.com/questions/97777/variablity-in-cv-glmnet-results
### cycle for doing 100 cross validations
### and take the average of the mean error curves
### initialize vector for final data.frame with Mean Standard Errors
#MSEs <- NULL
#for (i in 1:100){
#                 cv <- cv.glmnet(y, x, alpha=alpha, nfolds=k)  
#                 MSEs <- cbind(MSEs, cv$cvm)
#             }
#  rownames(MSEs) <- cv$lambda
#  lambda.min <- as.numeric(names(which.min(rowMeans(MSEs))))




#Five-fold cross-validation was simulated 100 times to estimate prediction model performance. 
#The final model was comprised of one regularized logistic regression model trained on a
#balanced training dataset, 
#classifying between high-risk and low-risk Multiple Myeloma.




# https://neptune.ai/blog/balanced-accuracy
# balanced accuracy = (sensitivty + specificity) / 2

# how to plot ROC curve


Find a gene list fitting the cox model
------------------------------------------------------

"The principal components or penalized Cox regression method is employed to select a gene list for fitting the Cox proportional harzards model. It provides an assessment of whether the association of expression data to survival data is statistically significant. It also lets the user evaluate whether the expression data provides more accurate predictions than that provided by standard clinical covariates."

https://stats.stackexchange.com/questions/411804/how-to-choose-the-best-combination-of-covariates-in-cox-multiple-regression
Identifying small sets of genes whose expression is related to outcome can readily be done with LASSO. It would select a subset of genes that together are strongly related to outcome while penalizing their regression coefficients to minimize overfitting. The R glmnet package provides tools for analyzing Cox models, including cross-validation to choose the optimum penalty factor (based on partial likelihood deviance in Cox models) and thus the number of genes chosen. LASSO can't avoid the dependence of the gene set chosen on the particular sample at hand, and you still should consider incorporating standard clinical variables into your model, but it provides a principled and well accepted approach to this type of problem

elastic net might be alos ok
https://accio.github.io/machine-learning/2018/01/04/glmnet.html







In [ ]:
y_annot = annot_data_m[,c("D_PFS","D_OS_FLAG")]
colnames(y_annot) = c("time", "status")
rownames(y_annot) = annot_data_m$Patient

x_geneexpr = t(nzv_gexp)
colnames(x_geneexpr) = paste0("gene",colnames(x_geneexpr))

x_geneexpr.age.iss = cbind(annot_data_m[,c("D_Age","D_ISS")], x_geneexpr)

#head(annot_df)
#x_geneexpr.age.iss[1:3, 1:3]
as.matrix(x_geneexpr.age.iss)[1:3, 1:3]


In [ ]:

# remove patients that ISS is NA
x_geneexpr.age.iss.nona = x_geneexpr.age.iss[-which(is.na(x_geneexpr.age.iss$D_ISS)),]
y_annot.nona = y_annot[-which(is.na(x_geneexpr.age.iss$D_ISS)),]

#a = apply(x_geneexpr.age.iss.nona, 1, function(x) which(is.na(x)))
#b = apply(x_geneexpr.age.iss.nona, 2, function(x) which(is.na(x)))



In [ ]:
#head(x_geneexpr.age.iss.nona)
#a
#b

In [ ]:
library(glmnet)
library(survival)

fit.elasticnet.noclinvar <- glmnet(as.matrix(x_geneexpr), as.matrix(y_annot), family = "cox", alpha=0.5) #TODO:alpha=1 by default
fit.lasso.noclinvar <- glmnet(as.matrix(x_geneexpr), as.matrix(y_annot), family = "cox", alpha = 1)


In [ ]:
fit.elasticnet.age.iss <- glmnet(x_geneexpr.age.iss.nona, as.matrix(y_annot.nona), family = "cox", alpha=0.5)


In [ ]:
fit.lasso.age.iss <- glmnet(x_geneexpr.age.iss.nona, as.matrix(y_annot.nona), family = "cox", alpha = 1)


In [ ]:
plot(fit.elasticnet.noclinvar, xvar = "lambda", label = TRUE)


head(coef(fit.elasticnet.noclinvar, s = 0.05))


In [ ]:
plot(fit.lasso.noclinvar, label=TRUE)


In [ ]:
plot(fit.elasticnet.age.iss, label=TRUE)


In [ ]:
plot(fit.lasso.age.iss, label=TRUE)


#library(biomaRt)
#mart <- useMart(dataset="hsapiens_gene_ensembl",biomart='ensembl')
#gene_names = biomaRt::select(mart, 
#                             keys=c("148"),
#                             columns=c('hgnc_symbol'), 
#                             keytype='entrezgene_id')
#gene_names
# ADRA1A https://pubmed.ncbi.nlm.nih.gov/30061158/

In [ ]:
set.seed(123)
cvfit.lasso.noclinvar <- cv.glmnet(x_geneexpr, as.matrix(y_annot), family = "cox", type.measure = "C", alpha = 1)
cvfit.elasticnet.noclinvar <- cv.glmnet(x_geneexpr, as.matrix(y_annot), family = "cox", type.measure = "C", alpha = 0.5)

cvfit.lasso.age.iss <- cv.glmnet(x_geneexpr.age.iss.nona, as.matrix(y_annot.nona), 
                                family = "cox", type.measure = "C", alpha = 1)
cvfit.elasticnet.age.iss <- cv.glmnet(x_geneexpr.age.iss.nona, as.matrix(y_annot.nona), 
                                 family = "cox", type.measure = "C", alpha = 0.5)



In [ ]:

print("cvfit.lasso.noclinvar")
plot(cvfit.lasso.noclinvar )
print("cvfit.elasticnet.noclinvar")
plot(cvfit.elasticnet.noclinvar)

print("cvfit.lasso.age.iss")
plot(cvfit.lasso.age.iss)

print("cvfit.elasticnet.age.iss")
plot(cvfit.elasticnet.age.iss)


In [ ]:
cvfit.lasso.noclinvar$lambda.min
cvfit.elasticnet.noclinvar$lambda.min


In [ ]:
# http://stats.stackexchange.com/questions/124288/r-glmnet-cross-validated-auc
        # cv.glmnet fits a whole sequence of models, and will report the auc for all of them. 
        # The max of the cvm sequence is the best model's auc.
        # cvm = mean cross-validated error - a vector of length length(lambda)
#        auc.val = max(model$models[[3]]$cvm)

In [ ]:
# https://github.com/bswhite/Celgene-Multiple-Myeloma-Challenge-Baseline-Models/blob/master/publishedClassifiers/lasso.weighted.ensemble/scratch.R

folds <- cvFolds(n=nrow(all.training.uams),K=10)
# Function to minimize to identify the best-fit alpha value
train <- function(par,numeric.matrix,folds,classes,mc) {
  library(glmnet)
  library(doParallel)
  registerDoParallel(cl = mc)
  model <- cv.glmnet(y=classes,x=numeric.matrix,family = 'cox',foldid = folds$which,parallel = T)
  model$cvm[model$lambda == model$lambda.1se]
}
mc <- makeCluster(detectCores())
classes <- as(Surv(time=as.numeric(all.training.uams$PFStimeMonths),event=all.training.uams$Progression),'matrix')
numeric.matrix=as.matrix(all.training.uams[,-c(1,2)])
optimized <- optim(par=0,train,method='Brent',lower=0,upper=1,numeric.matrix=numeric.matrix,folds=folds,classes=classes,mc)
model <- cv.glmnet(x=numeric.matrix,y=classes,family='cox',alpha=optimized$par,foldid = folds$which,parallel = T)
stopCluster(mc)
coef <- predict(model,newx=numeric.matrix,s=model$lambda.1se,type='coefficients')



# https://github.com/bswhite/Celgene-Multiple-Myeloma-Challenge-Baseline-Models/blob/master/publishedClassifiers/common/classifier-base.R
## Assume data is gene x samples.  This standardizes the genes so that each row has
## mean 0 and variance 1.
standardize.data <- function(data) {
    means <- rowMeans(data)
    sds <- apply(data,1,sd)
    data <- ((data - means) / sds)
    data
}



fit.and.threshold.coxph.model <- function(y, clinical.covariates) {

    ## Fit a Cox proportional hazards model
    surv.formula <- as.formula(paste("Surv(time, event)", "~",
                                     paste(clinical.covariates, collapse = " + "), sep = " "))
    cph.fit <- coxph(surv.formula, data = y)

    pred <- unname(predict(cph.fit, newdata = y, type = "risk"))
    y$cph <- pred

    ## Calculate a threshold for the Cox proportional hazards model                                                                                                                     
    suppressPackageStartupMessages(library(maxstat))
    mt.formula <- as.formula(paste("Surv(time, event)", "~", "cph", sep = " "))
    mt <- maxstat.test(mt.formula, data=data.frame(y), smethod="LogRank", pmethod="none")

    list(y = y, fitted.model = cph.fit, threshold = as.numeric(mt$estimate))

}



Classification via logistic regression
----------------------------------------


In [ ]:
# Its a top performing method

# The RNA-Seq data was first normalized using log transformation, 
# and both the microarray and RNA-Seq data were next standardized using z-scores for all genes.


# 1) genes mapped to certain pathways and chromosomal abnormalities
#    I. Chromosomal abnormalities [3]: deletion of chromosome 1p, 
#                                        gain of chromosome 1q, 
#                                        gain of chromosome 9, 
#                                        deletion of chromosome 13q, 
#                                        deletion of chromosome 17p, 
#                                        translocation t(4;14), 
#                                        trainslocation t(11;14), 
#                                        translocation t(14;16/14;20).
#    II. DNA repair pathways [5]: non-homologous end-joining pathway
#                                 homologous recombination pathway, 
#                                 Fanconi anemia pathway, 
#                                 nucleotide excision repair pathway, 
#                                 mismatch repair pathway, 
#                                 base excision repair pathway.
#    III. Other pathways [3]: cell cycle pathway, 
#                              p53 signalling pathway, 
#                              NF-kB signalling pathway, 
#                              Ras-ERK pathway.
#    IV. Genes targeted by Multiple Myeloma treatments [6]: Bortezomib, Thalidomide -> AURKA, IGF1,  AURKA (3), IGF1R (5), and FGFR3
#    V. Mutations associated with high-risk Multiple Myeloma [3].
#    VI. Other gene expression profiles obtained from literature: EMC92 [7], UAMS70 [8], 
#                       DNA repair pathway score [4], -> BUB1, BUB1B|PAK6, RAD51, PLK1, BRCA1, CENPA, BARD1, AURKA, MAD2L1, CENPH, XRCC2 and CDC25C|FAM53C)
#                       IFM group [9], -> TP53 
#                       cell death network [10].
# 2) gene assocaited with myeloma

# Sum of gene expressions were used, and feature selection of the engineered features 
# was also performed to progressively discard uninformative features. 
# We also include clinical data comprising age and ISS as features.


# Samples with missing data were discarded for the training dataset, 
# and only genes that were found across all training and validation datasets were retained. 
# To minimize the problem of class imbalance in training, random undersampling was performed to 
# ensure equal class distribution between high-risk and low-risk samples in the training dataset. 
# The machine learning algorithm chosen for the construction of the final prediction model was regularized 
# logistic regression, implemented using the glmnet package[11] in R. Five-fold cross-validation was simulated 
# 100 times to estimate prediction model performance. The final model was comprised of one regularized logistic 
# regression model trained on a balanced training dataset, classifying between high-risk and low-risk 
# Multiple Myeloma.


In [ ]:
# create the final model to be saved
    cv.glmn=cv.glmnet(x= data.list[[i]],
                      y=y.list[[i]],
                      family="binomial",
                      type.measure="auc",alpha=alpha)

A univariate approach to rank gene for feature selection in building expression based models
------------------------------------------------------------------

In [ ]:
# differential gene expession
# diff expr on gene expression doesnt make sense since I have only TPMs values
# https://support.bioconductor.org/p/98820/


In [ ]:
# Second Place Feature Selection Methods

# a simple univariate approach to rank gene for feature selection in building expression based models.
# In each of the four training datasets they computed each gene’s effect size, z, via the concordance
# index between overall survival and the gene’s expression. These effect sizes were then combined across
# training sets using Stouffer’s method with no weighting to yield a single meta-z per gene. They employed
# this meta concordance index method under two expression normalization procedures with CDKN3 and PHF19 
# coming out on top respectively and combined them with clinical features to create their model.
# For more information on SUGO’s model please see their full description in Synapse
# (https://www.synapse.org/#!Synapse:syn10380508/wiki/499377)


# https://www.biostars.org/p/344233/
# test each gene independently via Cox regression

library(survival)
library(RegParallel)

#print(dim(coxdata))
#exit()

#res_featuresel <- RegParallel(
   # data = coxdata,
   # formula = 'Surv(time, status) ~ [*]',
   # FUN = function(formula, data)
   #   coxph(formula = formula,
  #      data = data,
  #      ties = 'breslow',
 #       singular.ok = TRUE),
 #   FUNtype = 'coxph',
 #   variables = colnames(coxdata)[3:ncol(coxdata)],
#    blocksize = 2000,
#    cores = 3,
#    nestedParallel = FALSE,
#    conflevel = 95)

#saveRDS(res_featuresel,"./res_featuresel.RDS")
#res_featuresel = readRDS("./res_featuresel.RDS")

#res <- res[!is.na(res$P),]
#head(res_featuresel)

#res_featuresel <- res_featuresel[order(res_featuresel$LogRank, decreasing = FALSE),]
#final <- subset(res_featuresel, LogRank < 0.0001)
#final
#print(dim(coxdata))
#print(dim(final))
#head(final)
#final_entrez_ids = gsub("gene","",final$Variable)

#print(head(res_featuresel ))


In [ ]:
library(biomaRt)
mart <- useMart(dataset="hsapiens_gene_ensembl",biomart='ensembl')

#hgnc_symbol_ids = mclapply(final_entrez_ids, function(x){
#    biomaRt::select(mart, 
#                    keys=x,
#                    columns=c('hgnc_symbol'), 
#                    keytype='entrezgene_id')
#}, mc.cores=2)

gene_names = biomaRt::select(mart, 
                             keys=final_entrez_ids,
                             columns=c('hgnc_symbol'), 
                             keytype='entrezgene_id')
res_featuresel = cbind(res_featuresel, 
                       gene_name = gene_names)
head(res_featuresel)
#entrezgene_ids = as.character(rownames(expression_data))
#hgnc_symbol_ids = gene_names$hgnc_symbol
#rownames(expression_data) = hgnc_symbol_ids


In [ ]:
#head(res_featuresel)
#head(gene_names[,1])
#length(gene_names[,1])
#nrow(res_featuresel)
#head(res_featuresel)
#dim(res_featuresel)
#write.csv(gene_names,"./final_gene_names.csv")
#write.table(gene_names, "./final_gene_names.txt", append = FALSE, sep = " ", dec = ".",
#            row.names = FALSE, col.names = FALSE, quote = FALSE)

In [ ]:
#surv.formula <- as.formula(paste("Surv(time, status)", "~","1", sep = " "))
#print(surv.formula)
#cph.fit <- coxph(surv.formula, data = coxdata)
#head(cph.fit)

In [ ]:
#head(annot_data_m)
#annot_df = annot_data_m[,c("D_PFS","D_OS_FLAG", "D_Age", "D_ISS")]
#colnames(annot_df) = c("time", "status", "age", "ISS")
#coxdata <- cbind(annot_df, t(edata_m))
#colnames(coxdata) = as.character(colnames(coxdata))
#head(coxdata)

In [ ]:
# "EZH2","MCM4","TYMS","AURKB","CHEK1","CHEK1","ZWINT","CCNA2","BIRC5"
# "PHF19", "MMSET", "D_AGE", "D_ISS"

Survival modelling using Random Forest and Gradient Boosting
------------------------------------------------------------

Compare models
----------------------------

https://seandavi.github.io/TargetOsteoAnalysis/articles/multivariate_survival.html

In [ ]:
#surv_all = survival::coxph(
#  survival::Surv(time,status) ~ metastatic + CTA_chrX + CTA_all,
#  data=ssgsea_survival)
#anova(surv_all)

Assessing model performance
----------------------------------
The integrated area under the curve (iAUC) and balanced accuracy curve (BAC). While the AUC is a widely accepted metric of prediction accuracy, it is sensitive to the specific time threshold used to differentiate high and low patient risk.(https://www.nature.com/articles/s41375-020-0742-z#Tab1). The myeloma research community has not yet reached a consensus on the time point that best separates patients into risk groups, though there is a general agreement that it lies between 1 and 2 years for progression-free survival (PFS). We, therefore, chose the iAUC “centered” on 18 months as a more robust primary metric. The iAUC range began 6 months prior to 18 months and continued to 6 months past with a sliding PFS threshold moving from12 to 24 months at weekly increments. iAUCs computed in each validation cohort were combined into a weighted average (wiAUC) with each cohort iAUC weighted by the square of the number of high-risk patients in order to ensure larger studies with more high-risk patients did not overwhelm smaller studies while still weighting them more heavily.



In [ ]:
sessionInfo()